<a href="https://colab.research.google.com/github/adukhan98/Advanced-Voice-Chatbot/blob/main/VoiceChatbot_Using_Gradio%2C_Whisper%2C_OpenAi_and_eleven_Labs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Voice Chatbot with OpenAI API, Gradio and Whisper

## ⚙️ Install dependencies

In [ ]:
!pip install -U -q openai-whisper
!pip install -q gradio
!pip install -q openai
!pip install -q elevenlabs

## 🔌 Configure OpenAI and ElevenLabs Connection

In [ ]:
import openai
from getpass import getpass

openai.api_key = getpass('Enter your OpenAI API Key: ')

Enter your OpenAI API Key: ··········


In [ ]:
from elevenlabs import set_api_key
from getpass import getpass

set_api_key(getpass('Enter your ElevenLabs API Key: '))

Enter your ElevenLabs API Key: ··········


## 🔊 Initiate Whisper

In [ ]:
import whisper

model = whisper.load_model("base")

## 🚀 Main Code


In [ ]:
from typing import Optional


class Chat:

    def __init__(self, system: Optional[str] = None):
        self.system = system
        self.messages = []

        if system is not None:
            self.messages.append({
                "role": "system",
                "content": system
            })

    def prompt(self, content: str) -> str:
          self.messages.append({
              "role": "user",
              "content": content
          })
          response = chat.completions.create(
              model="gpt-3.5-turbo",
              messages=self.messages
          )
          response_content = response["choices"][0]["message"]["content"]
          self.messages.append({
              "role": "assistant",
              "content": response_content
          })
          return response_content

In [ ]:
from elevenlabs import generate, play

import gradio as gr


chat = Chat(system="You are a helpful assistant.")


def run_text_prompt(message, chat_history):
    bot_message = chat.prompt(content=message)

    audio = generate(
        text=bot_message,
        voice="Bella"
    )

    play(audio, notebook=True)

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history

    message_transcription = model.transcribe(audio)["text"]
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot()

    msg = gr.Textbox()
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio( type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(debug=True)